# Ejercicio de Pipelines

** Cargamos los datos, vamos a usar el [Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset)**

In [1]:
import pandas as pd
import numpy as np

pelis = pd.read_csv("data/movies.csv")

variable_objetivo = "ventas"

pelis.head()

,presupuesto,genero,lenguaje,popularidad,productores,pais,ventas,duracion,titulo,puntuacion,n_votos
0,NaN,Comedy,en,8.387519,Sandollar Productions,United States of America,76578911.0,106.0,Father of the Bride Part II,5.7,173.0
1,NaN,Drama,en,0.894647,Miramax,South Africa,676525.0,106.0,"Cry, the Beloved Country",6.7,13.0
2,3500000.0,Comedy,en,14.569650,New Line Cinema,United States of America,28215918.0,91.0,Friday,7.0,513.0
3,NaN,Comedy,en,8.963037,Paramount Pictures,United States of America,32.0,87.0,Black Sheep,6.0,124.0
4,12000000.0,Comedy,en,9.592265,Universal Pictures,United States of America,41205099.0,92.0,Happy Gilmore,6.5,767.0


### Crear Pipeline que seleccione las columnas numéricas (usando ColumnExtractor), impute (Imputer, con la media) y estandarice (StandardScaler) los datos numéricos

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer, StandardScaler

In [3]:
columnas_numericas = pelis.select_dtypes(np.number).columns
columnas_numericas

Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object')

In [4]:
from sklearn.base import TransformerMixin, BaseEstimator

class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns, output_type="matrix"):
        self.columns = columns
        self.output_type = output_type

    def transform(self, X, **transform_params):
        if isinstance(X, list):
            X = pd.DataFrame.from_dict(X)
        if self.output_type == "matrix":
            return X[self.columns].values
        elif self.output_type == "dataframe":
            return X[self.columns]
        raise Exception("output_type tiene que ser matrix o dataframe")

    def fit(self, X, y=None, **fit_params):
        return self  

In [5]:
pipeline_procesado = make_pipeline(
    ColumnExtractor(columns=columnas_numericas),
    Imputer(strategy="mean"),
    StandardScaler()
)

In [6]:
pipeline_procesado

Pipeline(memory=None,
     steps=[('columnextractor', ColumnExtractor(columns=Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object'),
        output_type='matrix')), ('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True))])

### Transforma el dataset. Imprime las dimensiones del output.

In [7]:
pelis_procesada = pipeline_procesado.fit_transform(pelis)
pelis_procesada.shape

(1344, 6)

In [8]:
pelis_procesada

array([[ -2.30934132e-16,   4.83540189e-01,   6.72892890e-01,
          1.83377052e-01,  -2.35105859e-01,  -1.75398320e-01],
       [ -2.30934132e-16,  -9.35413943e-01,  -5.84099030e-01,
          1.83377052e-01,   4.52409968e-01,  -5.00797371e-01],
       [ -8.79400265e-01,   1.65427423e+00,  -1.28029118e-01,
         -4.26257342e-01,   6.58664717e-01,   5.16074663e-01],
       ..., 
       [ -2.30934132e-16,  -1.08389423e+00,   0.00000000e+00,
         -6.29468807e-01,  -2.35105859e-01,  -5.21134812e-01],
       [ -2.30934132e-16,   7.40059020e-01,   0.00000000e+00,
         -4.66899635e-01,  -2.35105859e-01,  -4.01143912e-01],
       [ -2.30934132e-16,  -1.08174332e+00,   0.00000000e+00,
         -4.12470600e+00,  -2.88511106e-02,  -5.25202300e-01]])

### Crea un pipeline igual que el anterior pero que además tenga un estimador LinearRegression. Para la variable objetivo (**ventas**), calcula el MAE (mean absolute error) de este estimador (usando el pipeline). 

In [9]:
columnas_numericas

Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object')

In [10]:
columnas_numericas_independientes = pelis.drop(columns=variable_objetivo).select_dtypes(
                                                                            np.number).columns
columnas_numericas_independientes

Index(['presupuesto', 'popularidad', 'duracion', 'puntuacion', 'n_votos'], dtype='object')

In [11]:
from sklearn.linear_model import LinearRegression

estimador = LinearRegression()

pipeline_procesado = make_pipeline(
    ColumnExtractor(columns=columnas_numericas_independientes),
    Imputer(strategy="mean"),
    StandardScaler()
)

pipeline_estimador = make_pipeline(
    pipeline_procesado,
    estimador
)

In [12]:
from sklearn.model_selection import cross_val_score

resultados = cross_val_score(pipeline_estimador,
                            X=pelis,
                            y=pelis[variable_objetivo],
                            scoring="neg_mean_absolute_error")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Que raro, si estamos imputando los datos, no?

A no ser, que la variable objetivo tenga datos nulos!!

In [13]:
pelis[pelis[variable_objetivo].isnull()].shape

(141, 11)

Efectivamente, la variable objetivo (ventas) tiene valores inexistentes. Los eliminamos (que no imputamos, **no se puede imputar la variable objetivo!!**)

In [14]:
pelis_con_ventas = pelis[pelis[variable_objetivo].notnull()]

In [15]:
pelis_con_ventas.shape

(1203, 11)

In [16]:
resultados = cross_val_score(pipeline_estimador,
                            X=pelis_con_ventas,
                            y=pelis_con_ventas[variable_objetivo],
                            scoring="neg_mean_absolute_error",
                            cv=3)

In [17]:
resultados

array([-27777283.45806894, -23051662.88881182, -22526942.71129046])

In [18]:
resultados.mean()

-24451963.019390404